## Evaluating and Analyzing of XGBoost Trainined Models for Hospital Readmissions prediction

In this notebook, we are going to analyze a trained xgboost models to predict hospital readmissions for given patients based on their historical data.
Let's first start to import the packages needed.

In [1]:
import json
import os
import numpy as np
import pandas as pd
import shutil

from urllib.parse import urlparse

import boto3

import shap
import tarfile
import pickle

import matplotlib.pyplot as plt
#%matplotlib inline

pd.options.mode.chained_assignment = None

import xgboost as xgb

##User defined import
from metrics import compute_metrics

In [2]:
def copy_model_from_s3(s3_model_path, local_model_dir):
    """Copy model from s3 to local
    Args:
        s3_model_path(str): S3 path where the model gz is saved
    Returns:
        Destination model path
    """
    client = boto3.client('s3')
    o = urlparse(s3_model_path)
    bucket = o.netloc
    key = o.path
    key = key.lstrip('/')
    if not os.path.exists(local_model_dir): 
        os.makedirs(local_model_dir) 
    fname = os.path.basename(s3_model_path) 
    output_path = os.path.join(local_model_dir, fname)
    
    client.download_file(bucket, key, output_path)
    
    return output_path
   

def load_model(gz_model_path): 
    """
    Loads xgboost trained model from disk
    Args:
        gz_model_path(str): Compressed Model path
    Returns:
        xgboost: Xgboost model object
    """
    model_dir = os.path.dirname(gz_model_path)
    model_path = os.path.join(model_dir, 'xgboost-model')

    tar = tarfile.open(gz_model_path, "r:gz")
    tar.extractall(model_dir)
    tar.close()
    
    #Load Model
    model = pickle.load(open(model_path, "rb"))
    
    #Remove the local copy of the model files
    shutil.rmtree(model_dir)

    return model


def get_labels_scores(df_preds_labels, target_names=None):
    """Get labels and scores/predictions to compute model metrics
    Args:
        df_preds_labels(pd.DataFrame): Dataframe of predictions & true labels
        target_names(list): List of target events
    Returns:
        Tuple of labels(np.array), scores(np.array) and Event names(list)
    """
    labels = None
    scores = None
    if target_names is None:
        cols = df_preds_labels.columns.tolist()
        label_names = [col for col in cols if not col.endswith('_')]
        label_names = [name for name in label_names if not name.endswith('probs')]
        pred_names = [col for col in cols if col.endswith('probs')]
    else:
        label_names = target_names
        pred_names = [name+'_probs' for name in target_names]
    
    labels = df_preds_labels[label_names].values
    scores = df_preds_labels[pred_names].values

    return labels, scores, label_names


In [50]:
FOLDS = ['fold_'+str(i) for i in range(5)] + ['all']
FOLD_INDX = 0
CURRENT_FOLD = FOLDS[FOLD_INDX]
SPLIT = 'val'
NUM_FEATURES = 100
PREPROCESSED_DATA_DIR = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/re/1000/preprocessed/{}'.format(CURRENT_FOLD)
DATA_PATH = os.path.join(PREPROCESSED_DATA_DIR, SPLIT+'.csv')

TRAIN_DATA_DIR = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/re/1000/training/'
MODEL_DIR = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/re/1000/model/'

TRAIN_RESULTS_PATH = os.path.join(TRAIN_DATA_DIR, str(NUM_FEATURES), CURRENT_FOLD, 'train_results.csv')
FINAL_RESULTS_DIR = os.path.join(TRAIN_DATA_DIR, str(NUM_FEATURES), CURRENT_FOLD, 'final_results_positive')

Now, we will add all the values/paths needed to train the models

In [4]:
df_results = pd.read_csv(TRAIN_RESULTS_PATH)
df_results.head()

,class,num_features,val_auc,best_model_path
0,unplanned_readmission,100,0.6381,s3://cmsai-mrk-amzn/FinalData/RE/Models/XGBoos...


In [ ]:
# df_vis = df_results.pivot(index='num_features', columns='class', values='val_auc')
# df_vis.plot()

In [5]:
#Get models having the best performance for each target variable
idx = df_results.groupby('class')['val_auc'].transform(max) ==df_results['val_auc']
df_best = df_results[idx]
print(df_best.shape)
df_best.head()

(1, 4)


,class,num_features,val_auc,best_model_path
0,unplanned_readmission,100,0.6381,s3://cmsai-mrk-amzn/FinalData/RE/Models/XGBoos...


In [ ]:
# best_models = [['d_5990', 100, 0.7, 's3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/re/final/month-0/xgboost/2020-11-10-20-48-57/100/d_5990/output/sagemaker-xgboost-201110-2049-020-212dc74f/output/model.tar.gz'],
#                ['d_78605', 100, 0.6, 's3://cmsai-mrk-amzn/CSVModelInputs/Tes/models/re/final/month-0/xgboost/2020-11-10-20-48-57/100/d_5990/output/sagemaker-xgboost-201110-2049-016-3e3ab8f4/output/model.tar.gz']]
# columns = ['class', 'num_features', 'val_auc', 'best_model_path']
# df_best = pd.DataFrame(best_models, columns=columns)
# print(df_best.shape)
# df_best.head()

In [17]:
df_data = pd.read_csv(DATA_PATH)
print(df_data.shape)
df_data.head()

(312757, 301)


,h_99232,h_99233,h_71010,h_93010,h_99231,h_99223,h_A0425,h_99285,d_4280,d_4019,...,d_5693,h_81003,d_V0481,d_40390,d_71941,d_5183,h_76700,d_V7284,h_01402,unplanned_readmission
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
def get_model_predictions(row, df_data, local_model_dir):
    """Process the predictions and performance for best model for each class.
    df_data first column is labels and others are features
    """
    best_model_path = row['best_model_path']
    target = row['class']
    num_features = row['num_features']
    
    #Copy the best model from s3 to local
    output_path = copy_model_from_s3(best_model_path, local_model_dir)

    #Load the copied model
    model = load_model(output_path)
    
    preds = []
    features = df_data.columns.tolist()[:num_features]
    #Predict for data and save in pd Dataframe
    probs = model.predict(xgb.DMatrix(df_data[features].values, df_data[target].values))
    #probs = model.predict(xgb.DMatrix(df_data.iloc[:, :num_features], df_data[target].values, feature_names=feature_names))
    preds.append(df_data[target].tolist())
    preds.append((probs>=0.5).astype(int).tolist())
    preds.append(probs.tolist())
    
    columns = [target, target+'_', target+'_probs']
    return preds, columns


def get_all_predictions(df_best_models, df_data, local_model_dir):
    """Get predictions from each of the best models of each target variable."""
    num_rows = df_best_models.shape[0]
    all_columns = []
    all_preds = []
    for i in range(num_rows):
        row = df_best_models.iloc[i, :]
        preds, columns = get_model_predictions(row, df_data, local_model_dir)
        all_preds += preds
        all_columns += columns
        
    df_preds = pd.DataFrame(np.array(all_preds).T, columns=all_columns)
    return df_preds

Evaluate for a sample model

In [ ]:
# target = df_best.iloc[0, 0]
# num_features = df_best.iloc[0,1]
# best_model_path = df_best.iloc[0, 3]

# #Copy the best model from s3 to local
# output_path = copy_model_from_s3(best_model_path, MODEL_DIR)
# #Load the copied model
# model = load_model(output_path)
# #model.feature_names

# #Evaluate model on data
# feature_names = df_data.columns.tolist()[:num_features]
# auc = model.eval(xgb.DMatrix(df_data[feature_names].values, df_data[target].values))
# print('AUC: - {}'.format(auc))

In [ ]:
df_preds = get_all_predictions(df_best, df_data, MODEL_DIR)

In [ ]:
print(df_preds.shape)
df_preds.head()

In [ ]:
np_labels, np_scores, _ = get_labels_scores(df_preds)
target_names = df_best['class'].tolist()
df_metrics = compute_metrics(np_labels, np_scores, target_names=target_names)

In [ ]:
print('Labels Shape: {}, Scores Shape: {}'.format(np_labels.shape, np_scores.shape))
df_metrics.head()

In [ ]:
#pd.DataFrame(df_metrics.mean()).T

In [ ]:
# mn = df_metrics.min()
# mx = df_metrics.max()
# avg = df_metrics.mean()

# df_metrics.loc['Min'] = mn
# df_metrics.loc['Max'] = mx
# df_metrics.loc['Average'] = avg
# df_metrics.tail()

In [ ]:
feature_names = df_data.columns.tolist()[:NUM_FEATURES]
if not os.path.exists(FINAL_RESULTS_DIR):
    os.makedirs(FINAL_RESULTS_DIR)
    
#Save the features used
features_list_path = os.path.join(FINAL_RESULTS_DIR, 'features.txt')
with open(features_list_path, 'w') as fp:
    fp.write('\n'.join(feature_names))

#Save the final metrics results
final_results_path = os.path.join(FINAL_RESULTS_DIR, SPLIT+'_metrics.csv')
df_metrics.to_csv(final_results_path)

## Explainability and Visualization using SHAP (SHapley Additive exPlanations)

*Source: https://github.com/slundberg/shap*

In [8]:
import warnings
warnings.filterwarnings("ignore")

import shap
import matplotlib.pyplot as plt
%matplotlib inline

# load JS visualization code to notebook
#shap.initjs()

In [13]:
print(df_data.shape)
df_data.head()

(312757, 301)


,h_99232,h_99233,h_71010,h_93010,h_99231,h_99223,h_A0425,h_99285,d_4280,d_4019,...,d_5693,h_81003,d_V0481,d_40390,d_71941,d_5183,h_76700,d_V7284,h_01402,unplanned_readmission
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
DATA_PATH

'/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/re/1000/preprocessed/fold_0/val.csv'

In [15]:
raw_data_path = '/home/ec2-user/SageMaker/CMSAI/modeling/tes/data/final-global/re/1000/raw/fold_0/test/raw_test_data_1000_30days_anony.csv'
df_raw = pd.read_csv(raw_data_path, low_memory=False)
print(df_raw.shape)
df_raw.head()

(312757, 1004)


,999,998,997,996,995,994,993,992,991,990,...,5,4,3,2,1,0,unplanned_readmission,discharge_id,discharge_dt,patient_id
0,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,d_311,d_340,h_97530,h_99238,admission,discharge,False,040AF93LT_20110609,20110609,040AF93LT
1,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,d_2793,d_7822,d_99591,h_76857,h_99239,discharge,False,2MYFG4ASK_20110325,20110325,2MYFG4ASK
2,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,d_44101,d_74685,d_7823,h_99231,h_99239,discharge,False,HMI1PMYYY_20110515,20110515,HMI1PMYYY
3,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,h_99231,h_99232,1_days,d_25062,h_99238,discharge,False,7KG8UK4MP_20100218,20100218,7KG8UK4MP
4,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,d_5932,d_78650,h_76705,h_99238,p_8876,discharge,False,SAZFE3UFX_20110524,20110524,SAZFE3UFX


In [18]:
df_data['patient_id'] = df_raw['patient_id'].tolist()
print(df_data.shape)
df_data.head()

(312757, 302)


,h_99232,h_99233,h_71010,h_93010,h_99231,h_99223,h_A0425,h_99285,d_4280,d_4019,...,h_81003,d_V0481,d_40390,d_71941,d_5183,h_76700,d_V7284,h_01402,unplanned_readmission,patient_id
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,040AF93LT
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2MYFG4ASK
2,1,1,1,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,HMI1PMYYY
3,1,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,7KG8UK4MP
4,0,0,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,SAZFE3UFX


In [21]:
del df_raw

In [20]:
df_data['unplanned_readmission'].value_counts()

0    265945
1     46812
Name: unplanned_readmission, dtype: int64

In [24]:
df_data_pos = df_data[df_data['unplanned_readmission']==1]
print(df_data_pos.shape)
df_data_pos.head()

(46812, 302)


,h_99232,h_99233,h_71010,h_93010,h_99231,h_99223,h_A0425,h_99285,d_4280,d_4019,...,h_81003,d_V0481,d_40390,d_71941,d_5183,h_76700,d_V7284,h_01402,unplanned_readmission,patient_id
12,1,1,1,1,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,1,E8QL120D4
14,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,X0AUZR75S
16,1,0,1,1,1,1,0,1,0,1,...,0,0,0,0,0,1,0,0,1,QWDTBH8DG
17,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,3WYG12YJT
22,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,LWN938WW0


In [51]:
print('Processing explainability(shap) for {} data...'.format(SPLIT))

i = 0 
target = df_best.iloc[i, 0]
num_features = df_best.iloc[i, 1]
best_model_path = df_best.iloc[i, 3]

feature_names = df_data.columns.tolist()[:NUM_FEATURES]
X = df_data[feature_names]
X_pos = df_data_pos[feature_names]

y = df_data[target]
y_pos = df_data_pos[target]

#Create a new shap dir if not available
shap_dir = os.path.join(FINAL_RESULTS_DIR, 'shap_'+SPLIT)
if not os.path.exists(shap_dir):
    os.makedirs(shap_dir)

#Copy the best model from s3 to local
output_path = copy_model_from_s3(best_model_path, MODEL_DIR)
#Load the copied model
model = load_model(output_path)

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap_values_pos = explainer.shap_values(X_pos)

#Visualizing the shap value of the first 10 predictions of the positive examples
columns = df_data_pos.columns.tolist()
patient_id_idx = columns.index('patient_id')
for j in range(10):
    patient_id = df_data_pos.iloc[j, patient_id_idx]
    vis_path = os.path.join(shap_dir, 'shap_{}.png'.format(patient_id))
    shap.force_plot(explainer.expected_value, shap_values_pos[j,:], X_pos.iloc[j,:], matplotlib=True, show=False)
    plt.savefig(vis_path, bbox_inches='tight')
    plt.close("all")

shap_path = os.path.join(FINAL_RESULTS_DIR, 'shap_{}.csv'.format(SPLIT))
df_shap = pd.DataFrame(shap_values_pos, columns=feature_names)
df_shap['patient_id'] = df_data_pos['patient_id'].tolist()
columns = ['patient_id'] + feature_names
df_shap = df_shap[columns]
df_shap.to_csv(shap_path, index=False)
# # visualize the training set predictions
# #shap.force_plot(explainer.expected_value, shap_values, X) ## Out-of-memory Error

# # create a dependence plot to show the effect of a single feature across the whole dataset
# vis_path = os.path.join(shap_dir, target+'_per_feature_shap.png')
# shap.dependence_plot(feature_names[0], shap_values, X, show=False)
# plt.savefig(vis_path, bbox_inches='tight')
# plt.close("all")

# # summarize the effects of all the features
# shap.summary_plot(shap_values, X, show=False)
# vis_path = os.path.join(shap_dir, target+'_all_features_shap.png')
# plt.savefig(vis_path, bbox_inches='tight')
# plt.close("all")

#Compute the mean absolute value of the SHAP values for each feature to get a standard bar plot
print('Computing feature importance')
shap.summary_plot(shap_values, X, plot_type="bar", show=False)
vis_path = os.path.join(FINAL_RESULTS_DIR, 'feature_importance.png')
plt.savefig(vis_path, bbox_inches='tight')
plt.close("all")

# print('Shap Values and Visualizations Successfully Saved to {}!'.format(shap_dir))
print('Done!')

Processing explainability(shap) for val data...
Done!


In [49]:
print(df_shap.shape)
df_shap.head()

(46812, 101)


,patient_id,h_99232,h_99233,h_71010,h_93010,h_99231,h_99223,h_A0425,h_99285,d_4280,...,d_V5881,d_7862,d_99591,h_3120F,d_71945,d_4293,h_97110,d_78009,h_99215,d_7242
0,E8QL120D4,0.050493,0.060441,0.036523,-0.000111,0.056445,0.015198,0.068268,0.006177,-0.041911,...,-0.013891,-0.001464,-0.001400,-0.000665,0.001571,-0.002400,0.000465,0.002283,-0.003578,-0.004103
1,X0AUZR75S,-0.041313,-0.028552,-0.003063,0.004854,0.013197,0.025366,-0.048417,0.002571,-0.030960,...,-0.007155,0.002355,0.001773,0.003922,0.000970,-0.004989,0.002126,-0.000724,0.080926,-0.002729
2,QWDTBH8DG,0.025826,-0.070334,0.051104,-0.004951,0.030936,0.019530,-0.049842,0.050957,-0.043221,...,-0.008912,0.000549,0.000004,0.000529,0.000706,-0.004468,0.002849,-0.001913,-0.007448,-0.003399
3,3WYG12YJT,-0.019462,-0.006305,-0.062559,-0.023549,-0.014876,-0.015660,0.141052,-0.010102,-0.032424,...,-0.009895,-0.046937,-0.000964,0.002733,0.001751,-0.004938,0.000809,0.001918,-0.008486,-0.002130
4,LWN938WW0,-0.047594,-0.025359,-0.046395,0.002025,-0.021622,0.005767,-0.062986,0.010701,0.301444,...,-0.008510,0.001873,-0.000500,0.002418,0.000806,-0.004379,0.002013,0.000822,-0.002923,-0.001004


In [37]:
shap_values.min()

-0.6469054

In [43]:
#===========================================
#Original

In [ ]:
print('Processing for {} data...'.format(SPLIT))
feature_names = df_data.columns.tolist()[:NUM_FEATURES]
X = df_data[feature_names]

#Create a new shap dir if not available
shap_dir = os.path.join(FINAL_RESULTS_DIR, 'shap_'+SPLIT)
if not os.path.exists(shap_dir):
    os.makedirs(shap_dir)
    
num_rows = df_best.shape[0]
for i in range(num_rows):
    target = df_best.iloc[i, 0]
    num_features = df_best.iloc[i, 1]
    best_model_path = df_best.iloc[i, 3]

    y = df_data[target]

    #Copy the best model from s3 to local
    output_path = copy_model_from_s3(best_model_path, MODEL_DIR)
    #Load the copied model
    model = load_model(output_path)
    
    # explain the model's predictions using SHAP
    # (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    
    print('Computing SHAP Results for Target={}...'.format(target))
    
#     vis_path = os.path.join(shap_dir, target+'_shap_values.pkl')
#     with open(vis_path, 'wb') as fp:
#         pickle.dump(shap_values, fp)
        
    # visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
    vis_path = os.path.join(shap_dir, target+'_per_patient_shap.png')
    shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:], matplotlib=True, show=False)
    plt.savefig(vis_path, bbox_inches='tight')
    plt.close("all")
    
    # visualize the training set predictions
    #shap.force_plot(explainer.expected_value, shap_values, X) ## Out-of-memory Error
    
    # create a dependence plot to show the effect of a single feature across the whole dataset
    vis_path = os.path.join(shap_dir, target+'_per_feature_shap.png')
    shap.dependence_plot(feature_names[0], shap_values, X, show=False)
    plt.savefig(vis_path, bbox_inches='tight')
    plt.close("all")
    
    # summarize the effects of all the features
    shap.summary_plot(shap_values, X, show=False)
    vis_path = os.path.join(shap_dir, target+'_all_features_shap.png')
    plt.savefig(vis_path, bbox_inches='tight')
    plt.close("all")
    
    #Compute the mean absolute value of the SHAP values for each feature to get a standard bar plot
    shap.summary_plot(shap_values, X, plot_type="bar", show=False)
    vis_path = os.path.join(shap_dir, target+'_all_features_importance.png')
    plt.savefig(vis_path, bbox_inches='tight')
    plt.close("all")
    
print('Shap Values and Visualizations Successfully Saved to {}!'.format(shap_dir))